In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')

from typing import List

import numpy as np
import pyspark
import matplotlib.pyplot as plt
import seaborn as sns
import functools
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, isnan, when
from pyspark.sql.types import IntegerType, DoubleType, BooleanType
from pyspark.ml.feature import Imputer

In [2]:
spark = SparkSession.builder.appName("diabetes_indicators").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/11 22:31:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Load the Dataset

In [3]:
df_male = spark.read.csv("./data/diabetes/diabetes_male_data_with_missing.csv", header=True, inferSchema=True)
df_female = spark.read.csv("./data/diabetes/diabetes_female_data_with_missing.csv", header=True, inferSchema=True)

print(f"Number of records in Male: {df_male.count()}")
print(f"Number of records in Female: {df_female.count()}")

Number of records in Male: 111706
Number of records in Female: 141974


In [4]:
def unionAll(dfs: List):
    """ Function to merge two sources of data """
    unioned_df = functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)
    return unioned_df

In [5]:
df = unionAll([df_male, df_female])

In [6]:
assert df.columns == df_male.columns and df.columns == df_female.columns, "There has been a join error"